# GEP
https://arxiv.org/pdf/cs/0102027

## Components of GEP

- *Chromosome*: Linear string of symbols (genes) representing *functions* and *terminals* (e.g. a Single-Gene Chromosome: *+ a b \* c d*, where *+* is the root, *a* and *\** are children of *+*, while *b* and *c* are children of *\**)
<br> *Genes* are fixed-length strings, divided into two parts: *Heads* consists of *functions* and *terminals*, whereas *tails* contains only *terminals*. Essentially, the *head* generates any kind of *expression tree*, and the *tail* ensures that they have enough arguments to form a valid tree.  The number of genes is decided.
<br> The length of the *head* $h$ is chosen, and the length of the *tail* $t$ depends on it. $t = h(n-1)+1$, where $n$ is the maximum number of arguments for the function with most arguments (e.g. $n = 2$ for *+*).
<br> By default the linking function of different genes within a chromosome is chosen in advance, but the linking operation could also be introduced in the chromosome. 
- *Functions*: Operators s.a. +, -, *, /, or custom functions
- *Terminals*: Variables/Factors
- *Expression Tree (ET)*: Decoded form of the chromosome, used for evaluation (tree-like structure (phenotype)). Does not need to have the same size as the Chromosome.
- *Fitness Function*: Function to evaluate goodness of solution

## Steps
1. Create Initial Population: Random chromosomes
2. Decode Chromosomes to ETs
3. Evaluate Fítness of each individual
4. Decide whether to continue or stop (Termination Condition)
5. Select Surviving Individuals:
    - Elitism, to ensure that the top X individuals are part of the next generation (skipping the genetic operations) (and replicate them to the next generation).
    - Remaining population is subject to other selection procedures (e.g. roulette wheel) and genetic operations.
6. Apply Genetic Operations
    - Except for mutation, a chromosome can only be modified once per operator.
    - Mutation: Randomly change part of chromosome (each position in the chromosome has a mutation rate)
    <br> In the paper, they state that the typical *mutation rate* $p_m$ is two point mutations per chromosome $p_m = 2/positions\_in\_chromosome$. Every position has a $p_m$ chance to be mutated.
    - Transpositions: 
        - IS Transposition: Copy and insert randomly selected segment of chromosome into the head of any gene, at a random position other than the very start. The head of the gene is trimmed to maintain the fixed size.
    <br> Typically, an IS transposition rate $p_{is}$ of 0.1 is used and a set of 3 different lengths (e.g. [1,2,3]) are used. The start position of the IS element, its length and insertion point are selected randomly.
        - RIS Transposition: Copy and insert randomly selected segment of chromosome starting with a function to the root (start) of the head.
    <br> Typically the RIS transposition rate $p_{ris}$ is 0.1 and there is a set of 3 different possible IS element lengths. A point is randomly chosen in the head, and the gene is scanned downstream until a function is found. The last symbols of the head are lost (as many as the IS length)
        - Gene Transposition: Randomly selected gene (other than first) is moved (not copied!) to the beginning of the chromosome. Depending on the linking function, this may not have any impact on the chromosomes fitness, but it has an affect in crossover, allowing the duplication of genes. Typically, $p_g$ is 0.1.
    - Recombination: Every pair (after selection mechanism) has a chance to produce 2 off-springs, or otherwise proceed to the next generation. The overall recombination rate proposed in the paper is 0.7.
        - One-point Recombination: Chooses random position as crossover point, and exchange material at this point forming two offsprings. The probability $p_{1r}$ for this is usually the highest.
        - Two-point Recombination: Chooses 2 random positions as crossover points and exchanges material at this point. Probability is $p_{2r}$.
        - Gene Recombination: Chooses a random gene position and exchanges this one among the parents. Probability $p_{gr}$ is usually lower than the other two; frequently set to 0.1.
7. Repeat through generations from step 2 onwards

In [ ]:
import numpy as np

In [66]:
class GEP:
    def __init__(self, num_genes, head_length, function_map, function_arity, terminal_set, terminal_data): # todo : pass probabilities?
        self.num_genes = num_genes
        self.head_length = head_length
        self.tail_length = head_length * (np.max(list(function_arity.values())) - 1) + 1
        self.function_map = function_map
        self.function_arity = function_arity
        self.chromosome_length = num_genes * (head_length + self.tail_length)
        self.symbols_head = np.array(list(function_map.keys()) + terminal_set)
        self.symbols_tail = np.array(terminal_set)
        self.population = None
        self.fitness_scores = np.array([])
        self.terminal_data = terminal_data
        # compute gene split points
        self.gene_indices = np.array([(i * (head_length+self.tail_length), (i + 1) * (head_length+self.tail_length)) for i in range(self.num_genes)])
        # indices of head and tail
        self.head_indices = np.array([i for i in range(self.chromosome_length) if i % (head_length+self.tail_length) < head_length])
        self.tail_indices = np.array([i for i in range(self.chromosome_length) if i % (head_length+self.tail_length) >= head_length])
        

    # TODO: higher probability for function over terminal in heads
    def initialize_population(self, pop_size):
        # create random heads and tails for every gene
        heads = np.random.choice(self.symbols_head, size=(pop_size, self.num_genes, self.head_length))
        tails = np.random.choice(self.symbols_tail, size=(pop_size, self.num_genes, self.tail_length))
        # combine heads and tails for each gene and flatten along the gene axis
        self.population = np.concatenate((heads, tails), axis=2).reshape(pop_size, self.chromosome_length)
        return self.population

    # Decode a single chromosome into a list of matrices (one per gene, representing factors)
    def decode_chromosome_factors(self, chromosome):
        gene_matrices = []
        for g_start, g_end in self.gene_indices:
            gene_slice = chromosome[g_start:g_end]
            gene_matrix = self._decode_gene_postfix_conversion(gene_slice)
            gene_matrices.append(gene_matrix)
        return gene_matrices
    
    # Compute fitness for each chromosome in `population` by:
    # 1. Decoding each gene into its factors
    # 2. Passing all gene matrices to fitness_fn (parameter)
    # Done in chunks to avoid huge memory usage
    def evaluate_population(self, population, fitness_fn, chunk_size=100):
        pop_size = population.shape[0]
        fitness_scores = np.empty(pop_size, dtype=np.float64)
        
        for start in range(0, pop_size, chunk_size): # in chunks
            end = min(start + chunk_size, pop_size)
            for i in range(start, end):
                chromosome = population[i]
                gene_matrices = self.decode_chromosome_factors(chromosome)
                fitness = fitness_fn(gene_matrices)
                fitness_scores[i] = fitness
        return fitness_scores

    # Decode a single gene into a matrix of factors by converting the gene to postfix and evaluating it
    def _decode_gene_postfix_conversion(self, gene):
        postfix_expr = self._prefix_to_postfix(gene)
        return self._evaluate_postfix(postfix_expr)

    # Converts prefix gene (e.g. + A * B C) to postfix (e.g. A B C * +) to simplify evaluation step
    def _prefix_to_postfix(self, gene):
        idx = 0
        def convert():
            nonlocal idx
            symbol = gene[idx]
            idx += 1
            if symbol in self.function_arity:
                ar = self.function_arity[symbol]
                children = []
                for _ in range(ar):
                    children.extend(convert())
                children.append(symbol)
                return children
            else:
                return [symbol]
        return convert()

    # Evaluate a postfix expression. Each operator in self.function_map is expected to do in-place operations if possible
    def _evaluate_postfix(self, postfix_expr):
        stack = []
        for sym in postfix_expr:
            if sym in self.function_map:
                ar = self.function_arity[sym]
                # Pop arguments in reverse order so we apply them in the correct order.
                args = [stack.pop() for _ in range(ar)][::-1]
                result = self.function_map[sym](*args)
                stack.append(result)
            else:
                # Terminal symbol -> fetch the terminal_data matrix
                stack.append(self.terminal_data[sym])
        # By construction, exactly one item left
        return stack[0]
    
    # gets top n elite chromosomes; returning their indices
    def get_elite_indices(self, n):
        return np.argsort(self.fitness_scores)[-n:]
    
    # roulette wheel selection
    def _roulette_wheel_selection(self, population, fitness_scores, n):
        # Shift fitness scores to ensure all values are positive
        min_fitness = np.min(fitness_scores)
        if min_fitness < 0:
            shifted_fitness_scores = fitness_scores - min_fitness
        else:
            shifted_fitness_scores = fitness_scores
    
        total_fitness = np.sum(shifted_fitness_scores)
    
        # If total fitness is zero (e.g., all shifted scores are zero), assign equal probability
        if total_fitness == 0:
            probs = np.ones(len(population)) / len(population)
        else:
            probs = shifted_fitness_scores / total_fitness
    
        # Select indices based on probabilities
        return np.random.choice(len(population), size=n, p=probs)
    
    def _mutate_operator(self, population, mutation_rate):
        mutation_mask = np.random.rand(*population.shape) < mutation_rate
        mutated_population = population.copy()
        head_mask = mutation_mask[:, self.head_indices]
        tail_mask = mutation_mask[:, self.tail_indices]
        
        num_head_mut = head_mask.sum()
        num_tail_mut = tail_mask.sum()
        head_mutations = np.random.choice(self.symbols_head, size=num_head_mut)
        tail_mutations = np.random.choice(self.symbols_tail, size=num_tail_mut)
        mutated_population[:, self.tail_indices][tail_mask] = tail_mutations
        mutated_population[:, self.head_indices][head_mask] = head_mutations

        return mutated_population

    # IS Transposition: Copy a random segment from the head of a gene and insert it at a random position within the head
    # todo : asserts about length etc
    def _is_transposition(self, population, p_is=0.1, lengths=[1,2,3]):
        if self.head_length <= 1:
            return population
        mutated_pop = population.copy()
        pop_size = len(population)
    
        for idx in range(pop_size):
            # Decide if we apply IS transposition to this chromosome
            if np.random.rand() < p_is:
                # 1. Randomly choose the length of the transposon
                transposon_length = np.random.choice(lengths)
    
                # 2. Randomly choose a start position for the IS element in [0, chromosome_length - transposon_length]
                start_pos = np.random.randint(0, self.chromosome_length - transposon_length + 1)
    
                # 3. Randomly choose a gene in which we'll insert the transposon
                gene_target = np.random.randint(0, self.num_genes)
                g_start, g_end = self.gene_indices[gene_target]  # e.g., (start_index, end_index) of that gene in the chromosome
    
                # 4. Randomly choose an insertion site *within the head*, excluding the root
                #    So if the head is positions [g_start, ..., g_start+head_length-1],
                #    we can insert at any bond from (g_start+1) up to (g_start+head_length-1).
                insertion_site = np.random.randint(g_start+1, g_start+self.head_length-1)
    
                # 5. Extract the transposon from the original chromosome
                transposon = mutated_pop[idx, start_pos : start_pos + transposon_length]
    
                # 6. Extract the head slice we need to modify
                head_slice = mutated_pop[idx, g_start : g_start + self.head_length]
    
                # 7. Build the new head by inserting transpon at insertion_site of head_slice, and then trimming the new head
                i = insertion_site - g_start  # index within the head slice
                new_head = np.insert(head_slice, i, transposon)
                # trim new_head to have self.head_length length
                new_head = new_head[:self.head_length]
    
                # 8. Store back the new head into the mutated population
                mutated_pop[idx, g_start : g_start + self.head_length] = new_head
    
        return mutated_pop

    # RIS Transposition
    def _ris_transposition(self, population, p_ris=0.1, lengths=[1,2,3]):
        if self.head_length <= 1:
            return population
        mutated_pop = population.copy()
        pop_size = len(mutated_pop)
    
        for idx in range(pop_size):
            if np.random.rand() < p_ris:
                # 1. Randomly choose which gene to modify
                gene_target = np.random.randint(0, self.num_genes)
                g_start, g_end = self.gene_indices[gene_target]  # the slice of that gene in the chromosome
    
                # 2. Randomly choose a starting position in the HEAD to scan for a function
                # Get indices of any function symbol in the head
                function_indices = [i for i in range(g_start, g_start + self.head_length) if mutated_pop[idx, i] in self.function_arity]
                if len(function_indices) == 0:
                    continue
                # Choose a random function position
                function_pos = np.random.choice(function_indices)

                # 3. Randomly choose the transposon length
                transposon_length = np.random.choice(lengths)
    
                # 4. Extract the transposon
                transposon = mutated_pop[idx, function_pos : function_pos + transposon_length]
    
                # 5. Rebuild the head: place 'transposon' at the root, then take the original head minus the last transposon_length
                head_slice = mutated_pop[idx, g_start : g_start + self.head_length]
                # Example: new_head = transposon + all_but_last(transposon_length) of the old head
                new_head = np.concatenate([transposon, head_slice[:-transposon_length]])
    
                # 6. Store new head in the chromosome
                mutated_pop[idx, g_start : g_start + self.head_length] = new_head
    
        return mutated_pop

    # Gene Transposition
    def _gene_transposition(self, population, p_g=0.1):
        if self.num_genes < 2:
            return population
        
        mutated_pop = population.copy()
        pop_size = len(mutated_pop)
    
        # Each gene has length = head_length + tail_length
        single_gene_length = self.head_length + self.tail_length
    
        for idx in range(pop_size):
            if np.random.rand() < p_g:
                # 1. Randomly pick a gene index in [1 .. num_genes-1]
                #    (because the first gene cannot transpose to the start)
                gene_target = np.random.randint(1, self.num_genes)
    
                # 2. Extract the transposed gene
                chromosome = mutated_pop[idx]
                start = gene_target * single_gene_length
                end   = (gene_target + 1) * single_gene_length
                transposed_gene = chromosome[start:end]
    
                # 3. Rebuild the chromosome by:
                #    - Putting the transposed gene first
                #    - Then appending all other genes in original order, except the transposed one
                new_order_genes = []
                for g in range(self.num_genes):
                    if g == gene_target:
                        continue  # we already extracted it
                    gstart = g * single_gene_length
                    gend   = (g + 1) * single_gene_length
                    new_order_genes.append(chromosome[gstart:gend])
    
                # Build final chromosome
                new_chromosome = np.concatenate([transposed_gene] + new_order_genes)
                mutated_pop[idx] = new_chromosome
    
        return mutated_pop

    def _one_point_crossover(self, parent1, parent2):
        cut = np.random.randint(1, self.chromosome_length)  # in [1..chrom_len-1]
    
        # Create empty offspring
        child1 = np.empty_like(parent1)
        child2 = np.empty_like(parent2)
    
        # Fill slices
        child1[:cut] = parent1[:cut]
        child1[cut:] = parent2[cut:]
        child2[:cut] = parent2[:cut]
        child2[cut:] = parent1[cut:]
    
        return child1, child2

    def _two_point_crossover(self, parent1, parent2):
        # Choose two distinct cuts in [1..chromosome_length], ensuring cut1 < cut2
        cut_points = np.random.choice(range(1, self.chromosome_length), size=2, replace=False)
        cut1, cut2 = min(cut_points), max(cut_points)
    
        child1 = np.empty_like(parent1)
        child2 = np.empty_like(parent2)
    
        # child1
        child1[:cut1] = parent1[:cut1]
        child1[cut1:cut2] = parent2[cut1:cut2]
        child1[cut2:] = parent1[cut2:]
    
        # child2
        child2[:cut1] = parent2[:cut1]
        child2[cut1:cut2] = parent1[cut1:cut2]
        child2[cut2:] = parent2[cut2:]
    
        return child1, child2

    def _gene_crossover(self, parent1, parent2):
        child1 = parent1.copy()
        child2 = parent2.copy()
    
        if self.num_genes > 1:
            # Pick any gene index from [0..num_genes-1]
            gene_idx = np.random.randint(0, self.num_genes)
    
            g_start, g_end = self.gene_indices[gene_idx]  # e.g. (start, end) for that gene
            child1[g_start:g_end], child2[g_start:g_end] = parent2[g_start:g_end], parent1[g_start:g_end]
    
        # If there's only 1 gene, gene recombination doesnt do anything, 
        return child1, child2

    def _recombine_population(self, population, p1r=0.5, p2r=0.3, pgr=0.1):
        pop_size = len(population)
        new_pop = np.empty_like(population)
        indices = np.arange(pop_size)
        np.random.shuffle(indices)
    
        i = 0
        while i < pop_size:
            if i == pop_size - 1:
                # If there's an odd number, the last one just gets copied
                new_pop[i] = population[indices[i]]
                i += 1
            else:
                # Pick two parents
                p1 = population[indices[i]]
                p2 = population[indices[i+1]]
    
                # Decide which operator to apply
                r = np.random.rand()
                if r < p1r:
                    c1, c2 = self._one_point_crossover(p1, p2)
                elif r < p1r + p2r:
                    c1, c2 = self._two_point_crossover(p1, p2)
                elif r < p1r + p2r + pgr:
                    c1, c2 = self._gene_crossover(p1, p2)
                else:
                    # No crossover
                    c1, c2 = p1, p2
    
                new_pop[i]   = c1
                new_pop[i+1] = c2
                i += 2
    
        return new_pop


    def run(self, fitness_fn, num_generations, n_elite=1, population=None):
        if population is None and self.population is not None:
            population = self.population
        elif population is None:
            raise ValueError("No population provided")

        for gen in range(num_generations):
            # 1. Evaluate population fitness
            fitness_scores = self.evaluate_population(population, fitness_fn)
            self.fitness_scores = fitness_scores
            # 2. Select elite chromosomes
            elite_indices = self.get_elite_indices(n_elite)
            elite_chromosomes = population[elite_indices].copy()
            top_score = fitness_scores[elite_indices[-1]]
            print(f"Generation {gen}: Top score = {top_score}, Avg. Score = {np.mean(fitness_scores)},Uniques: {len(np.unique(population, axis=0))}")
            # 3. Select remaining population (roulette wheel selection)
            remaining_indices = self._roulette_wheel_selection(population, fitness_scores, len(population) - n_elite)
            selected_population = population[remaining_indices]
            # 4. Apply genetic operations        
            # 4.1: Mutation
            selected_population = self._mutate_operator(selected_population, mutation_rate=2/self.chromosome_length)  # TODO: pass mutation rate
            # 4.2: IS Transposition
            selected_population = self._is_transposition(selected_population, p_is=0.1, lengths=[1,2,3]) # TODO: rate
            # 4.3: RIS Transposition
            selected_population = self._ris_transposition(selected_population, p_ris=0.1, lengths=[1,2,3]) # TODO: rate
            # 4.4: Gene Transposition
            selected_population = self._gene_transposition(selected_population, p_g=0.1) # TODO: rate
            # 4.5: Recombination 
            selected_population = self._recombine_population(selected_population, p1r=0.5, p2r=0.3, pgr=0.1) # rate

            # 5. Combine elites and modified selected population
            population = np.vstack((elite_chromosomes, selected_population))

        self.population = population
        self.fitness_scores = self.evaluate_population(population, fitness_fn)
        # Print best fitness
        top_score = np.max(self.fitness_scores)
        print(f"Generation {num_generations}: Top score = {top_score}")
        return self.population, self.fitness_scores


# todo: function probability in head ?
# todo: early stop ?

In [67]:
# Define the test parameters
num_genes = 5
head_length = 12
terminal_set = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t"]
rows, cols = 10, 10  # Shape of the terminal matrices

# Randomly generate terminal data
terminal_data = {t: np.random.rand(rows, cols) for t in terminal_set}

# Define function map and arity
function_map = {
    '+': np.add,
    '-': np.subtract,
    '*': np.multiply,
    '/': lambda x, y: np.divide(x, y, where=y != 0)  # Safe division
}
function_arity = {op: 2 for op in function_map.keys()}

# Create a GEP instance
gep = GEP(
    num_genes=num_genes,
    head_length=head_length,
    function_map=function_map,
    function_arity=function_arity,
    terminal_set=terminal_set,
    terminal_data=terminal_data
)

# Initialize population
population = gep.initialize_population(pop_size=5000)

def my_fitness(gene_mats):
    fitness_scores = []
    for mat in gene_mats:
        # Get the diagonal elements
        diagonals = np.diag(mat)

        # Calculate the absolute difference of diagonal elements from 3
        diag_diff = np.abs(diagonals - 3)

        # Lower difference means higher fitness, so take the negative mean of differences
        fitness = -np.mean(diag_diff)
        fitness_scores.append(fitness)
    return np.max(fitness_scores)

# Compute fitness without storing giant 4D arrays
fitness_scores = gep.evaluate_population(population, my_fitness, chunk_size=100)
print("Fitness array shape:", fitness_scores.shape)  # (5000,)

Fitness array shape: (5000,)


In [68]:
gep.run(my_fitness, num_generations=500, n_elite=1)

Generation 0: Top score = -0.49252102371324735, Avg. Score = -2.2630886000504185,Uniques: 5000
Generation 1: Top score = -0.49252102371324735, Avg. Score = -2.06472138932632,Uniques: 4976
Generation 2: Top score = -0.4679419930168482, Avg. Score = -1.8243528441283692,Uniques: 4967
Generation 3: Top score = -0.4070058179769038, Avg. Score = -1.6617142433433718,Uniques: 4988
Generation 4: Top score = -0.4070058179769038, Avg. Score = -1.5146078194743329,Uniques: 4981
Generation 5: Top score = -0.4070058179769038, Avg. Score = -1.366677827226696,Uniques: 4972
Generation 6: Top score = -0.37006522126968644, Avg. Score = -1.240722172061787,Uniques: 4981
Generation 7: Top score = -0.37006522126968644, Avg. Score = -1.1124333648057956,Uniques: 4975
Generation 8: Top score = -0.37006522126968644, Avg. Score = -1.0173484619673356,Uniques: 4978
Generation 9: Top score = -0.37006522126968644, Avg. Score = -0.935033465875723,Uniques: 4981
Generation 10: Top score = -0.37006522126968644, Avg. Score

KeyboardInterrupt: 

In [65]:
# get best individual in gep
best_idx = np.argmax(gep.fitness_scores)
best_chromosome = gep.population[best_idx]
best_gene_matrices = gep.decode_chromosome_factors(best_chromosome)

# print diagionals of each gene matrix
for i, gene_mat in enumerate(best_gene_matrices):
    print(f"Gene {i}: Diagonals = {np.diag(gene_mat)}")

Gene 0: Diagonals = [ 1.43774566 22.989627   15.59300947  1.71891408  4.13696855  2.14459196
  6.31617961  5.15349563  1.39858448  1.67708897]
Gene 1: Diagonals = [3. 3. 3. 3. 3. 3. 3. 3. 3. 3.]
Gene 2: Diagonals = [2.53828689 1.90506902 4.97195062 2.05205095 2.41141653 2.44184478
 2.27305189 2.65145891 2.9094547  2.42483554]
Gene 3: Diagonals = [2.26844247 3.87542778 2.34731916 3.698762   2.43402628 2.88937715
 3.30463828 2.80664735 2.99417792 3.21287069]
Gene 4: Diagonals = [1.6038225  1.06133225 1.22565672 1.83410247 1.60359376 1.31921971
 1.22388951 1.64744609 1.77486042 1.74853389]


In [48]:
best_chromosome

array(['-', 'b', 'b', 'n', 'p', 'd', 'a', 't', 'n', 'n', 'k', 't', 'b',
       '-', 'b', 'b', 'b', '/', 'f', 'i', 'd', 'n', 'j', 'p', 's', 'k'],
      dtype='<U1')

In [42]:
terminal_data['i']

array([[0.80464972, 0.2712393 , 0.53385051, ..., 0.87395948, 0.03730511,
        0.3637563 ],
       [0.70109819, 0.78556057, 0.62628699, ..., 0.63322515, 0.72582081,
        0.27902717],
       [0.23865945, 0.95092816, 0.80910451, ..., 0.99327796, 0.33267144,
        0.30858437],
       ...,
       [0.97283561, 0.04326924, 0.28503301, ..., 0.85194795, 0.82479672,
        0.08919556],
       [0.33134839, 0.23808793, 0.27569265, ..., 0.61340001, 0.80152394,
        0.73722098],
       [0.01509411, 0.05205039, 0.39067578, ..., 0.30376099, 0.33924842,
        0.01799993]])

In [27]:
total_fitness = np.sum(gep.fitness_scores)
total_fitness

inf

In [30]:
terminal_data

{'a': array([[0.83196063, 0.24195141, 0.3501143 , ..., 0.60200419, 0.05729169,
         0.51850456],
        [0.28179158, 0.32704339, 0.72664784, ..., 0.89050144, 0.68626553,
         0.51906637],
        [0.16513028, 0.34382202, 0.18259215, ..., 0.13032023, 0.5470618 ,
         0.7878814 ],
        ...,
        [0.7082026 , 0.36263473, 0.59161755, ..., 0.77267118, 0.61678542,
         0.97958721],
        [0.46257204, 0.50355766, 0.66926338, ..., 0.53712608, 0.46120605,
         0.98605685],
        [0.01926004, 0.30963178, 0.78500804, ..., 0.97714759, 0.93198963,
         0.29548246]]),
 'b': array([[0.84507017, 0.47165164, 0.33104392, ..., 0.31348495, 0.65261478,
         0.02769616],
        [0.83558125, 0.21241216, 0.35946927, ..., 0.49635253, 0.14987193,
         0.816081  ],
        [0.98930386, 0.37961134, 0.39933734, ..., 0.921744  , 0.8825109 ,
         0.93485359],
        ...,
        [0.29892553, 0.0661743 , 0.6457127 , ..., 0.97954782, 0.4354595 ,
         0.06618408],
  

In [18]:
gep.fitness_scores

array([1.38221718e+23, 1.23658531e+06, 6.37167268e+09, ...,
       8.67021191e+43, 4.85115523e+44, 3.48894343e+49])